## Imports

In [1]:
from pymongo import MongoClient
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
import math

## Connect to mongo db

In [2]:
# Connect to Mongo server
host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Connect to mongo db
db = client.#database name here

## Define an embed() function

In [3]:
def embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection):
    
    # Get the distinct values in the parent collection that point to the child collection's entries of interest
    linking_attribute_values_in_parent_collection = db[parent_collection].find( {}, {linking_attribute_parent:1, '_id':0}).distinct(linking_attribute_parent)

    # Iterate over them
    for value in tqdm(linking_attribute_values_in_parent_collection):

        # If there is a nan, skip it
        if not math.isnan(value):
            
            # Find the respective documents in the child collection (that will be embedded)
            to_embed = list(db[child_collection].find( {linking_attribute_child:value}, {'_id':0} ))

            # If only one document will be embedded, remove the list around it
            # If multiple documents will be embedded, they will be interted as a list
            if len(to_embed) == 1:
                to_embed = to_embed[0]

            # Embed them in the parent collection's respective entries
            db[parent_collection].update_many( {linking_attribute_parent:value}, {'$set': {new_attribute:to_embed} } )
            
            # Commented out because descreases usability in certain situations:
            # Remove ID field from parent collection
            #db[parent_collection].update_many( {linking_attribute_parent:value}, {'$unset': { linking_attribute_parent:'' } } )

## Embed Application_Countries in Application_StateProvinces

In [4]:
linking_attribute_parent = 'CountryID'
linking_attribute_child = 'CountryID'
new_attribute = 'Country'
parent_collection = 'Application_StateProvinces'
child_collection = 'Application_Countries'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/1 [00:00<?, ?it/s]

[{'Country': {'Continent': 'North America',
              'CountryID': 230,
              'CountryName': 'United States',
              'CountryType': 'UN Member State',
              'FormalName': 'United States of America',
              'IsoAlpha3Code': 'USA',
              'IsoNumericCode': 840,
              'LatestRecordedPopulation': 313973000,
              'Region': 'Americas',
              'Subregion': 'Northern America'},
  'CountryID': 230,
  'LatestRecordedPopulation': 5437278,
  'SalesTerritory': 'Southeast',
  'StateProvinceCode': 'AL',
  'StateProvinceID': 1,
  'StateProvinceName': 'Alabama',
  '_id': ObjectId('60a9208b4b95d601fc64696e')}]


## Embed Application_StateProvinces in Application_Cities

In [5]:
linking_attribute_parent = 'StateProvinceID'
linking_attribute_child = 'StateProvinceID'
new_attribute = 'StateProvince'
parent_collection = 'Application_Cities'
child_collection = 'Application_StateProvinces'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/53 [00:00<?, ?it/s]

[{'CityID': 1,
  'CityName': 'Aaronsburg',
  'LatestRecordedPopulation': 613,
  'Location': '0xE6100000010C07E11B542C73444087C09140035D53C0',
  'StateProvince': {'Country': {'Continent': 'North America',
                                'CountryID': 230,
                                'CountryName': 'United States',
                                'CountryType': 'UN Member State',
                                'FormalName': 'United States of America',
                                'IsoAlpha3Code': 'USA',
                                'IsoNumericCode': 840,
                                'LatestRecordedPopulation': 313973000,
                                'Region': 'Americas',
                                'Subregion': 'Northern America'},
                    'CountryID': 230,
                    'LatestRecordedPopulation': 13284753,
                    'SalesTerritory': 'Mideast',
                    'StateProvinceCode': 'PA',
                    'StateProvinceID': 39,
     

## Embed Application_Cities in Purchasing_Suppliers (both DeliveryCity and PostalCity)

In [6]:
linking_attribute_parent = 'DeliveryCityID'
linking_attribute_child = 'CityID'
new_attribute = 'DeliveryCity'
parent_collection = 'Purchasing_Suppliers'
child_collection = 'Application_Cities'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

linking_attribute_parent = 'PostalCityID'
linking_attribute_child = 'CityID'
new_attribute = 'PostalCity'
parent_collection = 'Purchasing_Suppliers'
child_collection = 'Application_Cities'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

[{'AlternateContactPersonID': 22,
  'BankAccountBranch': 'Woodgrove Bank Zionsville',
  'BankAccountCode': '356981',
  'BankAccountName': 'A Datum Corporation',
  'BankAccountNumber': '8575824136',
  'BankInternationalCode': '25986',
  'DeliveryAddressLine1': 'Suite 10',
  'DeliveryAddressLine2': '183838 Southwest Boulevard',
  'DeliveryCity': {'CityID': 38171,
                   'CityName': 'Zionsville',
                   'LatestRecordedPopulation': 14160,
                   'Location': '0xE6100000010CDE115F37B6F9434031276893C39055C0',
                   'StateProvince': {'Country': {'Continent': 'North America',
                                                 'CountryID': 230,
                                                 'CountryName': 'United States',
                                                 'CountryType': 'UN Member '
                                                                'State',
                                                 'FormalName': 'United States '

## Embed Application_Cities in Sales_Customers (both DeliveryCity and PostalCity)

In [7]:
linking_attribute_parent = 'DeliveryCityID'
linking_attribute_child = 'CityID'
new_attribute = 'DeliveryCity'
parent_collection = 'Sales_Customers'
child_collection = 'Application_Cities'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

linking_attribute_parent = 'PostalCityID'
linking_attribute_child = 'CityID'
new_attribute = 'PostalCity'
parent_collection = 'Sales_Customers'
child_collection = 'Application_Cities'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/655 [00:00<?, ?it/s]

  0%|          | 0/655 [00:00<?, ?it/s]

[{'AccountOpenedDate': '2013-01-01',
  'AlternateContactPersonID': 1002,
  'BillToCustomerID': 1,
  'CustomerCategoryID': 3,
  'CustomerID': 1,
  'CustomerName': 'Tailspin Toys (Head Office)',
  'DeliveryAddressLine1': 'Shop 38',
  'DeliveryAddressLine2': '1877 Mittal Road',
  'DeliveryCity': {'CityID': 19586,
                   'CityName': 'Lisco',
                   'LatestRecordedPopulation': None,
                   'Location': '0xE6100000010CE73F5A52A4BF444010638852B1A759C0',
                   'StateProvince': {'Country': {'Continent': 'North America',
                                                 'CountryID': 230,
                                                 'CountryName': 'United States',
                                                 'CountryType': 'UN Member '
                                                                'State',
                                                 'FormalName': 'United States '
                                                         

## Embed Application_DeliveryMethods in Purchasing_PurchaseOrders

In [8]:
linking_attribute_parent = 'DeliveryMethodID'
linking_attribute_child = 'DeliveryMethodID'
new_attribute = 'DeliveryMethod'
parent_collection = 'Purchasing_PurchaseOrders'
child_collection = 'Application_DeliveryMethods'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/5 [00:00<?, ?it/s]

[{'ContactPersonID': 2,
  'DeliveryMethod': {'DeliveryMethodID': 2, 'DeliveryMethodName': 'Courier'},
  'DeliveryMethodID': 2,
  'ExpectedDeliveryDate': '2014-09-18',
  'IsOrderFinalized': 1,
  'OrderDate': '2014-08-29',
  'PurchaseOrderID': 1001,
  'SupplierID': 7,
  'SupplierReference': 'BC0280982',
  '_id': 1001}]


## Embed Application_DeliveryMethods in Sales_Invoices

In [9]:
linking_attribute_parent = 'DeliveryMethodID'
linking_attribute_child = 'DeliveryMethodID'
new_attribute = 'DeliveryMethod'
parent_collection = 'Sales_Invoices'
child_collection = 'Application_DeliveryMethods'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/1 [00:00<?, ?it/s]

[{'AccountsPersonID': 2001,
  'BillToCustomerID': 401,
  'ConfirmedDeliveryTime': '2013-02-10 08:40:00',
  'ConfirmedReceivedBy': 'Klara Rakus',
  'ContactPersonID': 2165,
  'CustomerID': 483,
  'CustomerPurchaseOrderNumber': '18172',
  'DeliveryInstructions': 'Suite 275, 593 Huq Avenue',
  'DeliveryMethod': {'DeliveryMethodID': 3,
                     'DeliveryMethodName': 'Delivery Van'},
  'DeliveryMethodID': 3,
  'DeliveryRun': '',
  'InvoiceDate': '2013-02-09',
  'InvoiceID': 1989,
  'IsCreditNote': 0,
  'OrderID': 2041,
  'PackedByPersonID': 7,
  'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for '
                          'collection","EventTime":"2013-02-09T12:00:00","ConNote":"EAN-125-3039"},{ '
                          '"Event":"DeliveryAttempt","EventTime":"2013-02-10T08:40:00","ConNote":"EAN-125-3039","DriverID":11,"Latitude":33.8487376,"Longitude":-82.2598460,"Status":"Delivered"}],"DeliveredWhen":"2013-02-10T08:40:00","ReceivedBy":"Klara '
                       

## Embed Application_DeliveryMethods in Purchasing_Suppliers

In [10]:
linking_attribute_parent = 'DeliveryMethodID'
linking_attribute_child = 'DeliveryMethodID'
new_attribute = 'DeliveryMethod'
parent_collection = 'Purchasing_Suppliers'
child_collection = 'Application_DeliveryMethods'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/5 [00:00<?, ?it/s]

[{'AlternateContactPersonID': 22,
  'BankAccountBranch': 'Woodgrove Bank Zionsville',
  'BankAccountCode': '356981',
  'BankAccountName': 'A Datum Corporation',
  'BankAccountNumber': '8575824136',
  'BankInternationalCode': '25986',
  'DeliveryAddressLine1': 'Suite 10',
  'DeliveryAddressLine2': '183838 Southwest Boulevard',
  'DeliveryCity': {'CityID': 38171,
                   'CityName': 'Zionsville',
                   'LatestRecordedPopulation': 14160,
                   'Location': '0xE6100000010CDE115F37B6F9434031276893C39055C0',
                   'StateProvince': {'Country': {'Continent': 'North America',
                                                 'CountryID': 230,
                                                 'CountryName': 'United States',
                                                 'CountryType': 'UN Member '
                                                                'State',
                                                 'FormalName': 'United States '

## Embed Application_DeliveryMethods in Sales_Customers

In [11]:
linking_attribute_parent = 'DeliveryMethodID'
linking_attribute_child = 'DeliveryMethodID'
new_attribute = 'DeliveryMethod'
parent_collection = 'Sales_Customers'
child_collection = 'Application_DeliveryMethods'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/1 [00:00<?, ?it/s]

[{'AccountOpenedDate': '2013-01-01',
  'AlternateContactPersonID': 1002,
  'BillToCustomerID': 1,
  'CustomerCategoryID': 3,
  'CustomerID': 1,
  'CustomerName': 'Tailspin Toys (Head Office)',
  'DeliveryAddressLine1': 'Shop 38',
  'DeliveryAddressLine2': '1877 Mittal Road',
  'DeliveryCity': {'CityID': 19586,
                   'CityName': 'Lisco',
                   'LatestRecordedPopulation': None,
                   'Location': '0xE6100000010CE73F5A52A4BF444010638852B1A759C0',
                   'StateProvince': {'Country': {'Continent': 'North America',
                                                 'CountryID': 230,
                                                 'CountryName': 'United States',
                                                 'CountryType': 'UN Member '
                                                                'State',
                                                 'FormalName': 'United States '
                                                         

## Embed Sales_CustomerCategories in Sales_Customers

In [12]:
linking_attribute_parent = 'CustomerCategoryID'
linking_attribute_child = 'CustomerCategoryID'
new_attribute = 'CustomerCategory'
parent_collection = 'Sales_Customers'
child_collection = 'Sales_CustomerCategories'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/5 [00:00<?, ?it/s]

[{'AccountOpenedDate': '2013-01-01',
  'AlternateContactPersonID': 1002,
  'BillToCustomerID': 1,
  'CustomerCategory': {'CustomerCategoryID': 3,
                       'CustomerCategoryName': 'Novelty Shop'},
  'CustomerCategoryID': 3,
  'CustomerID': 1,
  'CustomerName': 'Tailspin Toys (Head Office)',
  'DeliveryAddressLine1': 'Shop 38',
  'DeliveryAddressLine2': '1877 Mittal Road',
  'DeliveryCity': {'CityID': 19586,
                   'CityName': 'Lisco',
                   'LatestRecordedPopulation': None,
                   'Location': '0xE6100000010CE73F5A52A4BF444010638852B1A759C0',
                   'StateProvince': {'Country': {'Continent': 'North America',
                                                 'CountryID': 230,
                                                 'CountryName': 'United States',
                                                 'CountryType': 'UN Member '
                                                                'State',
                         

## Embed Purchasing_SupplierCategories in Purchasing_Suppliers

In [13]:
linking_attribute_parent = 'SupplierCategoryID'
linking_attribute_child = 'SupplierCategoryID'
new_attribute = 'SupplierCategory'
parent_collection = 'Purchasing_Suppliers'
child_collection = 'Purchasing_SupplierCategories'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/8 [00:00<?, ?it/s]

[{'AlternateContactPersonID': 22,
  'BankAccountBranch': 'Woodgrove Bank Zionsville',
  'BankAccountCode': '356981',
  'BankAccountName': 'A Datum Corporation',
  'BankAccountNumber': '8575824136',
  'BankInternationalCode': '25986',
  'DeliveryAddressLine1': 'Suite 10',
  'DeliveryAddressLine2': '183838 Southwest Boulevard',
  'DeliveryCity': {'CityID': 38171,
                   'CityName': 'Zionsville',
                   'LatestRecordedPopulation': 14160,
                   'Location': '0xE6100000010CDE115F37B6F9434031276893C39055C0',
                   'StateProvince': {'Country': {'Continent': 'North America',
                                                 'CountryID': 230,
                                                 'CountryName': 'United States',
                                                 'CountryType': 'UN Member '
                                                                'State',
                                                 'FormalName': 'United States '

## Embed Warehouse_StockGroups in Warehouse_StockItemStockGroups

In [14]:
linking_attribute_parent = 'StockGroupID'
linking_attribute_child = 'StockGroupID'
new_attribute = 'StockItemStockGroup'
parent_collection = 'Warehouse_StockItemStockGroups'
child_collection = 'Warehouse_StockGroups'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/9 [00:00<?, ?it/s]

[{'StockGroupID': 6,
  'StockItemID': 1,
  'StockItemStockGroup': {'StockGroupID': 6,
                          'StockGroupName': 'Computing Novelties'},
  'StockItemStockGroupID': 1,
  '_id': 1}]


## Embed Warehouse_StockItemStockGroups in Warehouse_StockItems

In [15]:
linking_attribute_parent = 'StockItemID'
linking_attribute_child = 'StockItemID'
new_attribute = 'StockGroup'
parent_collection = 'Warehouse_StockItems'
child_collection = 'Warehouse_StockItemStockGroups'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(db[parent_collection].find_one())

  0%|          | 0/227 [00:00<?, ?it/s]

{'Barcode': nan,
 'Brand': nan,
 'ColorID': nan,
 'InternalComments': nan,
 'IsChillerStock': False,
 'LeadTimeDays': 14,
 'MarketingComments': 'Complete with 12 projectiles',
 'OuterPackageID': 7,
 'Photo': nan,
 'QuantityPerOuter': 1,
 'RecommendedRetailPrice': 37.38,
 'SearchDetails': 'USB missile launcher (Green) Complete with 12 projectiles',
 'Size': nan,
 'StockGroup': [{'StockGroupID': 6,
                 'StockItemID': 1,
                 'StockItemStockGroup': {'StockGroupID': 6,
                                         'StockGroupName': 'Computing '
                                                           'Novelties'},
                 'StockItemStockGroupID': 1},
                {'StockGroupID': 1,
                 'StockItemID': 1,
                 'StockItemStockGroup': {'StockGroupID': 1,
                                         'StockGroupName': 'Novelty Items'},
                 'StockItemStockGroupID': 2},
                {'StockGroupID': 7,
                 'StockI

## Embed Application_PaymentMethods in Purchasing_SupplierTransactions

In [16]:
linking_attribute_parent = 'PaymentMethodID'
linking_attribute_child = 'PaymentMethodID'
new_attribute = 'PaymentMethod'
parent_collection = 'Purchasing_SupplierTransactions'
child_collection = 'Application_PaymentMethods'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/1 [00:00<?, ?it/s]

[{'AmountExcludingTax': 194225.7,
  'FinalizationDate': '2014-06-02',
  'IsFinalized': 1,
  'OutstandingBalance': 0.0,
  'PaymentMethod': {'PaymentMethodID': 4, 'PaymentMethodName': 'EFT'},
  'PaymentMethodID': 4,
  'PurchaseOrderID': 847,
  'SupplierID': 7,
  'SupplierInvoiceNumber': '3687',
  'SupplierTransactionID': 127482,
  'TaxAmount': 29133.86,
  'TransactionAmount': 223359.56,
  'TransactionDate': '2014-05-28',
  'TransactionTypeID': 5,
  '_id': 1001}]


## Embed Application_PaymentMethods in Sales_CustomerTransactions

In [17]:
linking_attribute_parent = 'PaymentMethodID'
linking_attribute_child = 'PaymentMethodID'
new_attribute = 'PaymentMethod'
parent_collection = 'Sales_CustomerTransactions'
child_collection = 'Application_PaymentMethods'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/1 [00:00<?, ?it/s]

[{'AmountExcludingTax': 0.0,
  'CustomerID': 1,
  'CustomerTransactionID': 231,
  'FinalizationDate': '2013-01-02',
  'InvoiceID': 41,
  'IsFinalized': 1,
  'OutstandingBalance': 0.0,
  'PaymentMethod': {'PaymentMethodID': 4, 'PaymentMethodName': 'EFT'},
  'PaymentMethodID': 4,
  'TaxAmount': 0.0,
  'TransactionAmount': -9513.49,
  'TransactionDate': '2013-01-02',
  'TransactionTypeID': 3,
  '_id': 42}]


## Embed Application_TransactionTypes in Purchasing_SupplierTransactions

In [18]:
linking_attribute_parent = 'TransactionTypeID'
linking_attribute_child = 'TransactionTypeID'
new_attribute = 'TransactionType'
parent_collection = 'Purchasing_SupplierTransactions'
child_collection = 'Application_TransactionTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/2 [00:00<?, ?it/s]

[{'AmountExcludingTax': 194225.7,
  'FinalizationDate': '2014-06-02',
  'IsFinalized': 1,
  'OutstandingBalance': 0.0,
  'PaymentMethod': {'PaymentMethodID': 4, 'PaymentMethodName': 'EFT'},
  'PaymentMethodID': 4,
  'PurchaseOrderID': 847,
  'SupplierID': 7,
  'SupplierInvoiceNumber': '3687',
  'SupplierTransactionID': 127482,
  'TaxAmount': 29133.86,
  'TransactionAmount': 223359.56,
  'TransactionDate': '2014-05-28',
  'TransactionType': {'TransactionTypeID': 5,
                      'TransactionTypeName': 'Supplier Invoice'},
  'TransactionTypeID': 5,
  '_id': 1001}]


## Embed Application_TransactionTypes in Sales_CustomerTransactions

In [19]:
linking_attribute_parent = 'TransactionTypeID'
linking_attribute_child = 'TransactionTypeID'
new_attribute = 'TransactionType'
parent_collection = 'Sales_CustomerTransactions'
child_collection = 'Application_TransactionTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/2 [00:00<?, ?it/s]

[{'AmountExcludingTax': 2300.0,
  'CustomerID': 832,
  'CustomerTransactionID': 2,
  'FinalizationDate': '2013-01-02',
  'InvoiceID': 1,
  'IsFinalized': 1,
  'OutstandingBalance': 0.0,
  'TaxAmount': 345.0,
  'TransactionAmount': 2645.0,
  'TransactionDate': '2013-01-01',
  'TransactionType': {'TransactionTypeID': 1,
                      'TransactionTypeName': 'Customer Invoice'},
  'TransactionTypeID': 1,
  '_id': 1}]


## Embed Application_TransactionTypes in Warehouse_StockItemTransactions

In [20]:
linking_attribute_parent = 'TransactionTypeID'
linking_attribute_child = 'TransactionTypeID'
new_attribute = 'TransactionType'
parent_collection = 'Warehouse_StockItemTransactions'
child_collection = 'Application_TransactionTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/3 [00:00<?, ?it/s]

[{'CustomerID': 832.0,
  'InvoiceID': 1.0,
  'PurchaseOrderID': nan,
  'Quantity': -10.0,
  'StockItemID': 67,
  'StockItemTransactionID': 1,
  'SupplierID': nan,
  'TransactionOccurredWhen': '2013-01-01 12:00:00.0000000',
  'TransactionType': {'TransactionTypeID': 10,
                      'TransactionTypeName': 'Stock Issue'},
  'TransactionTypeID': 10,
  '_id': 1}]


## Embed Warehouse_Colors in Warehouse_StockItems

In [21]:
linking_attribute_parent = 'ColorID'
linking_attribute_child = 'ColorID'
new_attribute = 'Color'
parent_collection = 'Warehouse_StockItems'
child_collection = 'Warehouse_Colors'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/8 [00:00<?, ?it/s]

[{'Barcode': nan,
  'Brand': nan,
  'Color': {'ColorID': 12, 'ColorName': 'Steel Gray'},
  'ColorID': 12.0,
  'InternalComments': nan,
  'IsChillerStock': False,
  'LeadTimeDays': 14,
  'MarketingComments': 'Complete with 12 projectiles',
  'OuterPackageID': 7,
  'Photo': nan,
  'QuantityPerOuter': 1,
  'RecommendedRetailPrice': 37.38,
  'SearchDetails': 'USB rocket launcher (Gray) Complete with 12 projectiles',
  'Size': nan,
  'StockGroup': [{'StockGroupID': 6,
                  'StockItemID': 2,
                  'StockItemStockGroup': {'StockGroupID': 6,
                                          'StockGroupName': 'Computing '
                                                            'Novelties'},
                  'StockItemStockGroupID': 4},
                 {'StockGroupID': 1,
                  'StockItemID': 2,
                  'StockItemStockGroup': {'StockGroupID': 1,
                                          'StockGroupName': 'Novelty Items'},
                  'StockItemS

## Embed Warehouse_PackageTypes in Warehouse_StockItems (both UnitPackage and OuterPackage)

In [22]:
linking_attribute_parent = 'UnitPackageID'
linking_attribute_child = 'PackageTypeID'
new_attribute = 'UnitPackageType'
parent_collection = 'Warehouse_StockItems'
child_collection = 'Warehouse_PackageTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

linking_attribute_parent = 'OuterPackageID'
linking_attribute_child = 'PackageTypeID'
new_attribute = 'OuterPackageType'
parent_collection = 'Warehouse_StockItems'
child_collection = 'Warehouse_PackageTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[{'Barcode': nan,
  'Brand': nan,
  'ColorID': nan,
  'InternalComments': nan,
  'IsChillerStock': False,
  'LeadTimeDays': 14,
  'MarketingComments': 'Complete with 12 projectiles',
  'OuterPackageID': 7,
  'OuterPackageType': {'PackageTypeID': 7, 'PackageTypeName': 'Each'},
  'Photo': nan,
  'QuantityPerOuter': 1,
  'RecommendedRetailPrice': 37.38,
  'SearchDetails': 'USB missile launcher (Green) Complete with 12 projectiles',
  'Size': nan,
  'StockGroup': [{'StockGroupID': 6,
                  'StockItemID': 1,
                  'StockItemStockGroup': {'StockGroupID': 6,
                                          'StockGroupName': 'Computing '
                                                            'Novelties'},
                  'StockItemStockGroupID': 1},
                 {'StockGroupID': 1,
                  'StockItemID': 1,
                  'StockItemStockGroup': {'StockGroupID': 1,
                                          'StockGroupName': 'Novelty Items'},
            

## Embed Warehouse_PackageTypes in Purchasing_PurchaseOrderLines

In [23]:
linking_attribute_parent = 'PackageTypeID'
linking_attribute_child = 'PackageTypeID'
new_attribute = 'PackageType'
parent_collection = 'Purchasing_PurchaseOrderLines'
child_collection = 'Warehouse_PackageTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/3 [00:00<?, ?it/s]

[{'Description': '"The Gu" red shirt XML tag t-shirt (White) XXS',
  'ExpectedUnitPricePerOuter': 84.0,
  'IsOrderLineFinalized': 1,
  'OrderedOuters': 1073,
  'PackageType': {'PackageTypeID': 6, 'PackageTypeName': 'Carton'},
  'PackageTypeID': 6,
  'PurchaseOrderID': 784,
  'PurchaseOrderLineID': 3001,
  'ReceivedOuters': 1073,
  'StockItemID': 77,
  '_id': 3001}]


## Embed Warehouse_PackageTypes in Sales_InvoiceLines

In [24]:
linking_attribute_parent = 'PackageTypeID'
linking_attribute_child = 'PackageTypeID'
new_attribute = 'PackageType'
parent_collection = 'Sales_InvoiceLines'
child_collection = 'Warehouse_PackageTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/3 [00:00<?, ?it/s]

[{'Description': '32 mm Double sided bubble wrap 50m',
  'ExtendedPrice': 3864.0,
  'InvoiceID': 978,
  'InvoiceLineID': 3001,
  'LineProfit': 1590.0,
  'PackageType': {'PackageTypeID': 7, 'PackageTypeName': 'Each'},
  'PackageTypeID': 7,
  'Quantity': 30,
  'StockItemID': 164,
  'TaxAmount': 504.0,
  'TaxRate': 15.0,
  'UnitPrice': 112.0,
  '_id': 3001}]


## Embed Warehouse_PackageTypes in Sales_OrderLines

In [25]:
linking_attribute_parent = 'PackageTypeID'
linking_attribute_child = 'PackageTypeID'
new_attribute = 'PackageType'
parent_collection = 'Sales_OrderLines'
child_collection = 'Warehouse_PackageTypes'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/3 [00:00<?, ?it/s]

[{'Description': '10 mm Double sided bubble wrap 50m',
  'OrderID': 682,
  'OrderLineID': 2001,
  'PackageType': {'PackageTypeID': 7, 'PackageTypeName': 'Each'},
  'PackageTypeID': 7,
  'PickedQuantity': 20,
  'PickingCompletedWhen': '2013-01-12 11:00:00',
  'Quantity': 20,
  'StockItemID': 158,
  'TaxRate': 15.0,
  'UnitPrice': 105.0,
  '_id': 2001}]


## Embed Purchasing_PurchaseOrderLines in Purchasing_PurchaseOrders

In [26]:
linking_attribute_parent = 'PurchaseOrderID'
linking_attribute_child = 'PurchaseOrderID'
new_attribute = 'PurchaseOrderLines'
parent_collection = 'Purchasing_PurchaseOrders'
child_collection = 'Purchasing_PurchaseOrderLines'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/2074 [00:00<?, ?it/s]

[{'ContactPersonID': 2,
  'DeliveryMethod': {'DeliveryMethodID': 2, 'DeliveryMethodName': 'Courier'},
  'DeliveryMethodID': 2,
  'ExpectedDeliveryDate': '2014-09-18',
  'IsOrderFinalized': 1,
  'OrderDate': '2014-08-29',
  'PurchaseOrderID': 1001,
  'PurchaseOrderLines': [{'Description': 'Shipping carton (Brown) '
                                         '305x305x305mm',
                          'ExpectedUnitPricePerOuter': 47.5,
                          'IsOrderLineFinalized': 1,
                          'OrderedOuters': 239,
                          'PackageType': {'PackageTypeID': 7,
                                          'PackageTypeName': 'Each'},
                          'PackageTypeID': 7,
                          'PurchaseOrderID': 1001,
                          'PurchaseOrderLineID': 3901,
                          'ReceivedOuters': 239,
                          'StockItemID': 184},
                         {'Description': 'Black and orange glass with care '
       

## Embed Sales_InvoiceLines in Sales_Invoices

In [27]:
linking_attribute_parent = 'InvoiceID'
linking_attribute_child = 'InvoiceID'
new_attribute = 'InvoiceLines'
parent_collection = 'Sales_Invoices'
child_collection = 'Sales_InvoiceLines'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/70510 [00:00<?, ?it/s]

[{'AccountsPersonID': 2001,
  'BillToCustomerID': 401,
  'ConfirmedDeliveryTime': '2013-02-10 08:40:00',
  'ConfirmedReceivedBy': 'Klara Rakus',
  'ContactPersonID': 2165,
  'CustomerID': 483,
  'CustomerPurchaseOrderNumber': '18172',
  'DeliveryInstructions': 'Suite 275, 593 Huq Avenue',
  'DeliveryMethod': {'DeliveryMethodID': 3,
                     'DeliveryMethodName': 'Delivery Van'},
  'DeliveryMethodID': 3,
  'DeliveryRun': '',
  'InvoiceDate': '2013-02-09',
  'InvoiceID': 1989,
  'InvoiceLines': [{'Description': 'Developer joke mug - Oct 31 = Dec 25 '
                                   '(White)',
                    'ExtendedPrice': 119.6,
                    'InvoiceID': 1989,
                    'InvoiceLineID': 6441,
                    'LineProfit': 68.0,
                    'PackageType': {'PackageTypeID': 7,
                                    'PackageTypeName': 'Each'},
                    'PackageTypeID': 7,
                    'Quantity': 8,
                    'Stock

## Embed Sales_OrderLines in SalesOrders

In [28]:
linking_attribute_parent = 'OrderID'
linking_attribute_child = 'OrderID'
new_attribute = 'OrderLines'
parent_collection = 'Sales_Orders'
child_collection = 'Sales_OrderLines'

embed(linking_attribute_parent, linking_attribute_child, new_attribute, parent_collection, child_collection)

pprint(list(db[parent_collection].find( {new_attribute: {"$exists": True} } ).limit(1)))

  0%|          | 0/73595 [00:00<?, ?it/s]

[{'BackorderOrderID': 1037,
  'ContactPersonID': 3012,
  'CustomerID': 812,
  'CustomerPurchaseOrderNumber': '16322',
  'ExpectedDeliveryDate': '2013-01-21',
  'IsUndersupplyBackordered': 1,
  'OrderDate': '2013-01-18',
  'OrderID': 1001,
  'OrderLines': [{'Description': 'USB food flash drive - fortune cookie',
                  'OrderID': 1001,
                  'OrderLineID': 3053,
                  'PackageType': {'PackageTypeID': 7,
                                  'PackageTypeName': 'Each'},
                  'PackageTypeID': 7,
                  'PickedQuantity': 7,
                  'PickingCompletedWhen': '2013-01-18 11:00:00',
                  'Quantity': 7,
                  'StockItemID': 14,
                  'TaxRate': 15.0,
                  'UnitPrice': 32.0},
                 {'Description': 'IT joke mug - that behavior is by design '
                                 '(White)',
                  'OrderID': 1001,
                  'OrderLineID': 3054,
                 